In [1]:
from policyengine_us import Simulation


situation = {
    "people": {
        "you": {"age": {"2024": 40}, "employment_income": {"2024": 2000000}}
    },
    "families": {"your family": {"members": ["you"]}},
    "marital_units": {"your marital unit": {"members": ["you"]}},
    "tax_units": {"your tax unit": {"members": ["you"]}},
    "spm_units": {"your household": {"members": ["you"]}},
    "households": {
        "your household": {"members": ["you"], "state_name": {"2024": "TX"}}
    },
}

simulation = Simulation(
    situation=situation,
)

output = simulation.calculate("household_net_income", 2024)
print(output)

/Users/pavelmakarchuk/anaconda3/envs/pe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[1251561.]


In [2]:
taxable_income = simulation.calculate("taxable_income", 2024)
print(taxable_income)

[1985400.]


In [3]:
amt = simulation.calculate("alternative_minimum_tax", 2024)
print(amt.sum())

0.0


In [4]:
tax_exempt_interest_income = simulation.calculate(
    "tax_exempt_interest_income", 2024
)
print(tax_exempt_interest_income)

[0.]


In [5]:
from policyengine_us import Simulation


situation = {
    "people": {
        "you": {
            "age": {"2024": 40},
            "real_estate_taxes": {"2024": 100000},
            "charitable_cash_donations": {"2024": 100000},
            "charitable_non_cash_donations": {"2024": 100000},
        }
    },
    "families": {"your family": {"members": ["you"]}},
    "marital_units": {"your marital unit": {"members": ["you"]}},
    "tax_units": {"your tax unit": {"members": ["you"]}},
    "spm_units": {"your household": {"members": ["you"]}},
    "households": {
        "your household": {"members": ["you"], "state_name": {"2024": "CA"}}
    },
    "axes": [
        [{"name": "employment_income", "count": 200, "min": 0, "max": 2000000}]
    ],
}

simulation = Simulation(
    situation=situation,
)

output = simulation.calculate("household_net_income", 2024)
print(output)

[   3495.      13614.745   20518.86    27655.145   36517.586   45396.99
   54225.97    63024.965   71823.96    80622.95    89421.95    98220.945
  107019.94   115818.94   124617.93   133101.78   141498.75   150035.5
  159055.61   168075.72   177086.77   186016.42   194946.06   203875.72
  212805.36   221735.     230664.66   239557.97   248336.64   257069.42
  265809.2    274503.47   283167.78   291832.1    298943.9    305660.56
  312210.3    318644.72   325056.     331467.25   337873.7    343480.94
  349088.16   354695.4    360302.62   365909.88   371259.5    376565.25
  381870.97   387176.7    392482.44   397788.16   403093.88   408399.62
  413705.3    419011.06   424316.78   429622.5    434928.25   440233.97
  445539.7    450845.44   456151.16   461456.88   466698.03   471897.22
  477096.44   482295.62   487494.8    492694.     497893.2    503003.38
  508096.     513188.7    518281.34   523374.03   528466.75   533559.4
  538652.     543744.7    548837.4    553930.     558927.25   563

In [6]:
amt = simulation.calculate("alternative_minimum_tax", 2024)
print(amt)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
